In [1]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms
import copy

import numpy as np
import pandas as pd

%matplotlib inline 
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import math
import string

import tqdm

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture

from skimage.filters import sobel
from skimage.color import rgb2gray

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [2]:
#This is the classifier Class.
from torchvision.transforms.transforms import RandomRotation, RandomAdjustSharpness, RandomGrayscale

class Classifier(torch.nn.Module):

  def __init__(self, backbone='resnet', multi_backbone = False, device ="cuda:0",dropout_rate = 0.2, do_augmentation = False):
    super().__init__()
    self.multi_backbone = multi_backbone # Bool: Indicates if we use multibackbone

    #In the following secttion we download the appropriatee prettrained model
    if backbone == "vgg19":
      backbone = torchvision.models.vgg19(pretrained=True)
      self.out_channels = 25088
      
    elif backbone == "resnet18":
      backbone = torchvision.models.resnet18(pretrained=True)
      self.out_channels = 512

    elif backbone == "resnet50":
      backbone = torchvision.models.resnet50(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b1":
      backbone = torchvision.models.efficientnet_b1(pretrained=True)
      self.out_channels = 1280

    elif backbone == "Efficientnet b3":
      backbone = torchvision.models.efficientnet_b3(pretrained=True)
      self.out_channels = 1536

    elif backbone == "Efficientnet b5":
      backbone = torchvision.models.efficientnet_b5(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b7":
      backbone = torchvision.models.efficientnet_b7(pretrained=True)
      self.out_channels = 2560
     
    modules = list(backbone.children())[:-1]
    self.do_augmentation = do_augmentation

    if self.multi_backbone: #We create the backbones and put them on the device
      self.backbone1 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone2 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone3 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone4 = nn.Sequential(*copy.deepcopy(modules)).to(device)

    else:
      self.backbone =  nn.Sequential(*modules).to(device)


    #This is the final classification layer
    self.fc = nn.Sequential(nn.Dropout(dropout_rate),
                            nn.Linear(self.out_channels * 4, 3))                
     
  def forward(self, x, is_training = True):
    # Transform image range from (0, 1) to (-1, 1)

    imgs = [x[:,i] for i in range(4)] #list of 4 images
    

    if self.multi_backbone:
      encodings = [self.backbone1(imgs[0]).flatten(1), 
                   self.backbone2(imgs[1]).flatten(1),
                   self.backbone3(imgs[2]).flatten(1),
                   self.backbone4(imgs[3]).flatten(1)]
    else:
      encodings = [self.backbone(img).flatten(1) for img in imgs]

    return self.fc(torch.cat(encodings,1))

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Instantiate the classifier using the following weights
# If your directory structure does not resemble the following, you may need to 
model = torch.load("drive/MyDrive/Fish Attribution/" + 'model1e-050.5.2022-05-11 13_52_21.pt', map_location="cpu")
model.eval()
model.zero_grad()

In [5]:
print(torch.cuda.is_available())
# check that cuda is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

False
cpu


In [6]:
# Load the raw fish images
# WARNING: The following WILL use a large amount of RAM
import numpy as np

X_PATH = "/content/drive/Shareddrives/Exploding Gradients/X_cropped_b.npy"
Y_PATH= "/content/drive/Shareddrives/Exploding Gradients/y_b.npy"

x = np.load(X_PATH)
y = np.load(Y_PATH)

# 285 instances, currently the shapes are off, see next cell for fix
print(x.shape)
print(y.shape)

(285, 4, 130, 750, 3)
(285, 1)


In [7]:
# Conver to PyTorch tensors from raw numpy arrays and
# fix the dimensions
tensor_x = torch.Tensor(x) 
tensor_y = torch.Tensor(y).long()

tensor_x = torch.swapaxes(tensor_x,2,4)
tensor_x = torch.swapaxes(tensor_x,3,4)

In [8]:
# There are 258 pieces of data, with 4 images per instance
# and each image being full color (RGB) with 130x750 pixels
# (The original fish images are of larger dimension but
# an autocropper was created to narrow down the focus)
tensor_x.shape

torch.Size([285, 4, 3, 130, 750])

In [9]:
tensor_y.shape

torch.Size([285, 1])

In [10]:
tensor_y[0]

tensor([1])

In [11]:
# Create a PyTorch dataloader to feed data to the model
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

attribution_ds = TensorDataset(tensor_x ,tensor_y) 

# Each time this is called, 40 pieces of data (40x4 = 160) are provided
# This was figured out in a bit of trial-and-error fashion because
# you have a limited about of onboard GPU RAM. 40 seems to be the maximum
# it's happy with and any Colab Pro instance goes into the red at this point
attribution_dl = DataLoader(attribution_ds,1,shuffle = True)

# Delete the raw numpy arrays from RAM, there's no need for them
# now that they exist as PyTorch tensors
del x,y

In [12]:
# Install Captum for Feature Attribution
!pip install -q captum

from captum.attr import *

In [13]:
model.eval()

Classifier(
  (augmentation): Sequential(
    (0): RandomHorizontalFlip(p=0.5)
    (1): RandomVerticalFlip(p=0.5)
    (2): RandomCrop(size=(130, 750), padding=None)
    (3): RandomRotation(degrees=[-10.0, 10.0], interpolation=bilinear, expand=False, fill=0)
  )
  (backbone1): Sequential(
    (0): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
              (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): Ad

In [14]:
lrp = LRP(model)

In [15]:
image, label = iter(attribution_dl).next()

In [16]:
image.shape

torch.Size([1, 4, 3, 130, 750])

In [17]:
label

tensor([[1]])

In [18]:
r = lrp.attribute(image, target=label)

TypeError: ignored

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
image.squeeze()[0].shape

In [ ]:
plt.imshow(image.squeeze()[1].permute(1,2,0).detach().numpy())

In [ ]:
from captum.attr import visualization as viz

_ = viz.visualize_image_attr(r.squeeze()[3].permute(1,2,0).detach().numpy(), image.squeeze()[3].permute(1,2,0).detach().numpy(), method="blended_heat_map", sign="all", alpha_overlay=1,
                          show_colorbar=True, title="Deconvolution")